In [1]:
import os
from PIL import Image, ImageOps
import numpy as np
from keras_preprocessing.sequence import pad_sequences

from keras.layers import Dense, LSTM, Reshape, BatchNormalization, Input, Conv2D, MaxPool2D, Lambda, Bidirectional
from keras.models import Model
import keras.backend as K
from keras.callbacks import ModelCheckpoint

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
filenames = []
for dirname, _, files in os.walk('D:\captcha\captcha\pics'):
  for f in files:
    filenames = np.append(filenames, f)
        
num_samples = len(filenames)
print('number of samples: ', num_samples)

number of samples:  1413


In [12]:
# list of our characters
char_list = "0123456789"

In [13]:
def encode_to_labels(txt):
    # encoding each label into list of digits
    encoded_list = []
    for char in txt:
        encoded_list.append(char_list.index(char))
    
    return encoded_list

In [20]:
path = r'D:/captcha/captcha/pics/'

# lists for training dataset
training_img = []                    # the images for training the model
training_txt = []                    # the labels
train_input_length = []              # the input of LSTM part of the model
train_label_length = []              # the label's length (4 to 7)
train_orig_txt = []
 
#lists for validation dataset
valid_img = []
valid_txt = []
valid_input_length = []
valid_label_length = []
valid_orig_txt = []
 
max_label_len = 0                    # max length for our labels (in this case 7)

In [21]:
for file in filenames:
    raw = Image.open(path + file)
    gray = ImageOps.grayscale(raw)
    img = np.array(gray)
    img = np.expand_dims(img , axis = 2)
    img = img/255.

    txt = file.split('.')[0]

    if len(txt) > max_label_len:
        max_label_len = len(txt)
    
    # split the dataset (85% train, 15% test)
    if np.random.rand() >= 0.85:
        valid_orig_txt.append(txt)
        valid_label_length.append(len(txt))
        valid_input_length.append(75)
        valid_img.append(img)
        valid_txt.append(encode_to_labels(txt))
    else:
        train_orig_txt.append(txt)
        train_label_length.append(len(txt))
        train_input_length.append(75)
        training_img.append(img)
        training_txt.append(encode_to_labels(txt))

In [22]:
print('number of training images: ', len(training_img))
print('number of validation images: ', len(valid_img))

number of training images:  89
number of validation images:  11


In [23]:
print(max_label_len)

7


In [24]:
# pad each output label to maximum text length
 
train_padded_txt = pad_sequences(training_txt, maxlen=max_label_len, padding='post', value = len(char_list))
valid_padded_txt = pad_sequences(valid_txt, maxlen=max_label_len, padding='post', value = len(char_list))

In [25]:
# input with shape of height=64 and width=306
inputs = Input(shape=(64,306,1))
 

conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)

conv_7 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_6)
batch_norm_7 = BatchNormalization()(conv_7)
pool_7 = MaxPool2D(pool_size=(2, 1))(batch_norm_7)

conv_8 = Conv2D(512, (2,2), activation = 'relu')(pool_7)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_8)
 

blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)

# this is the model we use for our prediction(after training)
prediction_model = Model(inputs, outputs)

In [26]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])

In [27]:
# this is the model for training
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')

In [28]:
print(model.summary())

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 306, 1)  0           []                               
                                ]                                                                 
                                                                                                  
 conv2d (Conv2D)                (None, 64, 306, 64)  640         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 32, 153, 64)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 32, 153, 128  73856       ['max_pooling2d[0][0]']    

In [29]:
training_img = np.array(training_img)
train_input_length = np.array(train_input_length)
train_label_length = np.array(train_label_length)

valid_img = np.array(valid_img)
valid_input_length = np.array(valid_input_length)
valid_label_length = np.array(valid_label_length)

training_txt = np.array(training_txt)
valid_txt = np.array(valid_txt)

batch_size = 2
epochs = 500

C:\Users\saeed\AppData\Local\Temp\ipykernel_21144\2322070578.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training_txt = np.array(training_txt)
C:\Users\saeed\AppData\Local\Temp\ipykernel_21144\2322070578.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  valid_txt = np.array(valid_txt)


In [30]:
model.fit(x=[training_img, train_padded_txt, train_input_length, train_label_length],
          y=np.zeros(len(training_img)),
          batch_size=batch_size,
          epochs = epochs,
          validation_data = ([valid_img, valid_padded_txt, valid_input_length, valid_label_length], [np.zeros(len(valid_img))]),
          verbose = 1)

Epoch 1/500
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
45/45 [==============================] - 15s 244ms/step - loss: 28.7365 - val_loss: 16.4963
Epoch 2/500
45/45 [==============================] - 10s 227ms/step - loss: 15.9696 - val_loss: 16.2254
Epoch 3/500
45/45 [==============================] - 11s 235ms/step - loss: 15.9226 - val_loss: 16.0287
Epoch 4/500
45/45 [==============================] - 11s 245ms/step - loss: 15.9097 - val_loss: 17.5205
Epoch 5/500
45/45 [==============================] - 11s 254ms/step - loss: 15.9455 - val_loss: 16.1620
Epoch 6/500
45/45 [==============================] - 11s 241ms/step - loss: 15.8549 - val_loss: 16.5196
Epoch 7/500
45/45 [==============================] - 11s 246ms/step - loss: 15.8678 - val_loss: 16.0898
Epoch 8/500
45/45 [==============================] - 11s 255ms/step - l

45/45 [==============================] - 16s 346ms/step - loss: 14.0495 - val_loss: 19.5365
Epoch 152/500
45/45 [==============================] - 16s 350ms/step - loss: 13.9126 - val_loss: 17.4431
Epoch 153/500
45/45 [==============================] - 13s 280ms/step - loss: 13.9972 - val_loss: 16.3044
Epoch 154/500
45/45 [==============================] - 13s 279ms/step - loss: 13.9489 - val_loss: 18.4009
Epoch 155/500
45/45 [==============================] - 14s 313ms/step - loss: 13.7723 - val_loss: 16.5338
Epoch 156/500
45/45 [==============================] - 14s 307ms/step - loss: 13.7748 - val_loss: 16.7164
Epoch 157/500
45/45 [==============================] - 12s 271ms/step - loss: 14.0868 - val_loss: 22.4878
Epoch 158/500
45/45 [==============================] - 12s 268ms/step - loss: 13.8457 - val_loss: 19.3785
Epoch 159/500
45/45 [==============================] - 14s 301ms/step - loss: 14.1197 - val_loss: 24.6175
Epoch 160/500
45/45 [==============================] - 14s 3

Epoch 306/500
45/45 [==============================] - 13s 296ms/step - loss: 8.4397 - val_loss: 18.0104
Epoch 307/500
45/45 [==============================] - 13s 279ms/step - loss: 8.1259 - val_loss: 20.1544
Epoch 308/500
45/45 [==============================] - 14s 303ms/step - loss: 10.8633 - val_loss: 26.4335
Epoch 309/500
45/45 [==============================] - 14s 321ms/step - loss: 10.1868 - val_loss: 25.2052
Epoch 310/500
45/45 [==============================] - 14s 317ms/step - loss: 9.5593 - val_loss: 33.4639
Epoch 311/500
45/45 [==============================] - 13s 279ms/step - loss: 11.1756 - val_loss: 126.9798
Epoch 312/500
45/45 [==============================] - 13s 282ms/step - loss: 11.2922 - val_loss: 18.3681
Epoch 313/500
45/45 [==============================] - 13s 290ms/step - loss: 9.5623 - val_loss: 20.9002
Epoch 314/500
45/45 [==============================] - 14s 318ms/step - loss: 9.9822 - val_loss: 18.5039
Epoch 315/500
45/45 [=============================

Epoch 384/500
45/45 [==============================] - 13s 281ms/step - loss: 3.4022 - val_loss: 23.7068
Epoch 385/500
45/45 [==============================] - 13s 299ms/step - loss: 3.4565 - val_loss: 24.2820
Epoch 386/500
45/45 [==============================] - 15s 324ms/step - loss: 3.1413 - val_loss: 23.2759
Epoch 387/500
45/45 [==============================] - 14s 302ms/step - loss: 3.3026 - val_loss: 22.4094
Epoch 388/500
45/45 [==============================] - 13s 280ms/step - loss: 3.0458 - val_loss: 25.1265
Epoch 389/500
45/45 [==============================] - 13s 296ms/step - loss: 3.0416 - val_loss: 22.8285
Epoch 390/500
45/45 [==============================] - 15s 324ms/step - loss: 3.2137 - val_loss: 26.2163
Epoch 391/500
45/45 [==============================] - 14s 320ms/step - loss: 3.6832 - val_loss: 24.6392
Epoch 392/500
45/45 [==============================] - 13s 281ms/step - loss: 3.8528 - val_loss: 23.4235
Epoch 393/500
45/45 [==============================] - 

45/45 [==============================] - 15s 328ms/step - loss: 0.8611 - val_loss: 31.0453
Epoch 463/500
45/45 [==============================] - 14s 313ms/step - loss: 1.1791 - val_loss: 33.9891
Epoch 464/500
45/45 [==============================] - 13s 285ms/step - loss: 2.1179 - val_loss: 28.8516
Epoch 465/500
45/45 [==============================] - 13s 289ms/step - loss: 1.7194 - val_loss: 30.4291
Epoch 466/500
45/45 [==============================] - 14s 316ms/step - loss: 1.2862 - val_loss: 28.8306
Epoch 467/500
45/45 [==============================] - 15s 331ms/step - loss: 1.2164 - val_loss: 28.9433
Epoch 468/500
45/45 [==============================] - 13s 292ms/step - loss: 1.5621 - val_loss: 29.6008
Epoch 469/500
45/45 [==============================] - 13s 288ms/step - loss: 2.1707 - val_loss: 31.6458
Epoch 470/500
45/45 [==============================] - 14s 310ms/step - loss: 2.8738 - val_loss: 28.8728
Epoch 471/500
45/45 [==============================] - 15s 333ms/step

In [31]:
model.save_weights('model_weights_V1_100data.hdf5')

In [32]:
# load the model weights
prediction_model.load_weights('model_weights_V1_100data.hdf5')

# predict outputs on validation images
prediction = prediction_model.predict(valid_img[:10])

# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])
 
# see the results
i = 0
for x in out:
    print("original_text =  ", valid_orig_txt[i])
    print("predicted text = ", end = '')
    for p in x:  
        if int(p) != -1:
            print(char_list[int(p)], end = '')       
    print('\n')
    i+=1

1/1 [==============================] - 1s 1s/step
original_text =   0891051
predicted text = 38450

original_text =   1168658
predicted text = 785528

original_text =   138073
predicted text = 40878

original_text =   227341
predicted text = 788570

original_text =   649788
predicted text = 5828

original_text =   69822
predicted text = 77411

original_text =   70209
predicted text = 10150

original_text =   8054393
predicted text = 8833926

original_text =   813317
predicted text = 2332070

original_text =   9682858
predicted text = 6238888

